<a href="https://colab.research.google.com/github/gyasifred/msc-thesis/blob/main/fine_turning__huggingface_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!  pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]
! pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 5.2 MB/s 
     |████████████████████████████████| 4.9 MB 63.0 MB/s 
     |████████████████████████████████| 116 kB 65.9 MB/s 
     |████████████████████████████████| 1.3 MB 61.7 MB/s 
     |████████████████████████████████| 212 kB 69.6 MB/s 
     |████████████████████████████████| 120 kB 69.5 MB/s 
     |████████████████████████████████| 115 kB 68.6 MB/s 
     |████████████████████████████████| 127 kB 72.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 5.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-no

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.environ["WANDB_DISABLED"]="true"

In [5]:
import transformers

print(transformers.__version__)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

4.22.1


In [6]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-tw"

In [7]:
from datasets import load_metric

In [8]:
# This code was adapted from https://github.com/GhanaNLP/kasa/blob/master/Kasa/Preprocessing.py
# A subclass of the kasafrench for preprocessing data
# import required library
import re
import unicodedata


class Preprocessing:
    # dummy initialization method
    def __init__(self):
        # initialize with some default parameters here later
        pass

    # read in parallel twi - english dataset
    def read_parallel_dataset(self, filepath_1, filepath_2, filepath_3=None):
        if filepath_3 != None:
            # read first language data
            lang_1 = []
            with open(filepath_1, encoding='utf-8') as file:
                line = file.readline()
                cnt = 1
                while line:
                    lang_1.append(line.strip())
                    line = file.readline()
                    cnt += 1

            # read second language data
            lang_2 = []
            with open(filepath_2, encoding='utf-8') as file:

                # twi=file.read()
                line = file.readline()
                cnt = 1
                while line:
                    lang_2.append(line.strip())
                    line = file.readline()
                    cnt += 1
            # Read third Language data
            lang_3 = []
            with open(filepath_3, encoding='utf-8') as file:
                line = file.readline()
                cnt = 1
                while line:
                    lang_3.append(line.strip())
                    line = file.readline()
                    cnt += 1

            return lang_1, lang_2, lang_3
            
        else:
            # read first language data
            lang_1 = []
            with open(filepath_1, encoding='utf-8') as file:
                line = file.readline()
                cnt = 1
                while line:
                    lang_1.append(line.strip())
                    line = file.readline()
                    cnt += 1

            # read second language data
            lang_2 = []
            with open(filepath_2, encoding='utf-8') as file:

                # twi=file.read()
                line = file.readline()
                cnt = 1
                while line:
                    lang_2.append(line.strip())
                    line = file.readline()
                    cnt += 1

            return lang_1, lang_2

    # Define a helper function to remove string accents

    def removeStringAccent(self, s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
        )

    # normalize input twi sentence
    def normalize_twi(self, s):
        s = self.removeStringAccent(s)
        s = s.lower()
        s = re.sub(r'([!.?])', r' \1', s)
        s = re.sub(r'[^a-zA-Z.ƆɔɛƐ!?’]+', r' ', s)
        s = re.sub(r'\s+', r' ', s)
        return s

    # normalize input french sentence
    def normalize_FrEn(self, s):
        s = self.removeStringAccent(s)
        s = s.lower()
        s = re.sub(r'([!.?])', r' \1', s)
        s = re.sub(r'[^a-zA-Z.!?]+', r' ', s)
        s = re.sub(r'\s+', r' ', s)
        return s
    
    def writeTotxt(self,destination,data):
        with open(destination, 'w') as f:
            for line in data:
                 f.write(f"{line}\n")

In [9]:
# import preprocessing class 
# Create an instance of tft preprocessing class
preprocessor = Preprocessing()

# Read raw parallel dataset
raw_data_twi, raw_data_en = preprocessor.read_parallel_dataset(
    filepath_1='/content/drive/MyDrive/verified_twi.txt',
    filepath_2='/content/drive/MyDrive/verified_english.txt'
)

# Normalize the raw data
raw_data_en = [preprocessor.normalize_FrEn(data) for data in raw_data_en]
raw_data_twi = [preprocessor.normalize_twi(data) for data in raw_data_twi]

In [10]:
# split the dataset into training and test sets
# Keep 10% of the data as test set
from sklearn.model_selection import train_test_split
train_twi,val_twi,train_en,val_en = train_test_split(raw_data_twi,raw_data_en, test_size=0.1, random_state=42)

In [11]:
import pandas as pd
translation=[]
for i,j in zip(train_en,train_twi):
  translation.append({"en":i,"tw":j})

train ={"translation":translation}
train=pd.DataFrame(train)

In [12]:
translation=[]
for i,j in zip(val_en,val_twi):
  translation.append({"en":i,"tw":j})

val ={"translation":translation}
val=pd.DataFrame(train)

In [13]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
print(train_dataset)
val_dataset = Dataset.from_pandas(val)
print(val_dataset)

Dataset({
    features: ['translation'],
    num_rows: 22878
})
Dataset({
    features: ['translation'],
    num_rows: 22878
})


In [14]:
# from datasets import load_dataset
# dataset = load_dataset("csv", data_files={"train": "train.csv", "val": "val.csv"})
# #from datasets import load_dataset
# #dataset = load_dataset("csv", data_files="my_file.csv")
# dataset

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/823k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [16]:
tokenizer("Hello, this one sentence!")

{'input_ids': [42, 38997, 2, 88, 108, 6496, 74, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
sentence = ['ɛsɛ sɛ w ankasa wo ne ɔkyerɛkyerɛfo no kasa .']
tokenizer(text_target=sentence)

{'input_ids': [[122, 8, 182, 172, 56, 12, 4245, 5, 273, 42, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [18]:
prefix = ""
max_input_length = 50
max_target_length = 50
source_lang = "en"
target_lang = "tw"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels= tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [19]:
tokenized_train_datasets = train_dataset.map(preprocess_function, batched=True)
tokenized_val_datasets =  val_dataset.map(preprocess_function, batched=True)

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/23 [00:00<?, ?ba/s]

In [20]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/296M [00:00<?, ?B/s]

In [21]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
metric = load_metric("sacrebleu")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [24]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_val_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 22878
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 57200


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.944400,0.643941,68.114500,10.569700


Saving model checkpoint to opus-mt-en-tw-finetuned-en-to-tw/checkpoint-500
Configuration saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-500/config.json
Model weights saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-500/special_tokens_map.json
Saving model checkpoint to opus-mt-en-tw-finetuned-en-to-tw/checkpoint-1000
Configuration saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-1000/config.json
Model weights saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-1000/tokenizer_config.json
Special tokens file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to opus-mt-en-tw-finetuned-en-to-tw/checkpoint-1500
Configuration saved i

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.944400,0.643941,68.114500,10.569700
2,0.760700,0.493727,75.525100,10.431500
3,0.625500,0.398112,79.736500,10.406000
4,0.501500,0.324665,82.600300,10.471700
5,0.425300,0.269130,84.689700,10.466200
6,0.377500,0.223624,86.275000,10.519300
7,0.327500,0.188932,88.257400,10.474900
8,0.284400,0.158729,89.706000,10.477300
9,0.240900,0.135139,90.792300,10.485000
10,0.217100,0.116405,91.797000,10.475300


Saving model checkpoint to opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5000
Configuration saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5000/config.json
Model weights saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5000/tokenizer_config.json
Special tokens file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5000/special_tokens_map.json
Deleting older checkpoint [opus-mt-en-tw-finetuned-en-to-tw/checkpoint-3500] due to args.save_total_limit
Saving model checkpoint to opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5500
Configuration saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5500/config.json
Model weights saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5500/tokenizer_config.json
Special tokens file saved in opus-mt-en-tw-finetuned-en-to-tw/checkpoint-5500/special_to

In [1]:
trainer.save_model("/content/drive/ opus-mt-en-tw-finetuned")

NameError: ignored